##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Classificação de texto com o TensorFlow Hub: avaliações de filmes

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/text_classification_with_hub"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/keras/text_classification_with_hub.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/keras/text_classification_with_hub.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tutorials/keras/text_classification_with_hub.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
  <td>     <a href="https://tfhub.dev/s?module-type=text-embedding"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">Ver modelos do TF Hub</a>
</td>
</table>

Este notebook classifica avaliações de filmes como *positivas* ou *negativas*, com base no texto da avaliação. Este é um exemplo de classificação *binária*, ou de duas classes, um tipo de problema de aprendizado de máquina importante, com diversas aplicações.

Este tutorial demonstra a aplicação básica do aprendizado por transferência usando o [TensorFlow Hub](https://tfhub.dev) e o Keras.

Usaremos o [dataset do IMDB](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb), que contém o texto de 50 mil avaliações de filmes do [Internet Movie Database](https://www.imdb.com/). Elas são divididas em 25 mil avaliações para treinamento e 25 mil para teste. Os conjuntos de treinamento e teste são *equilibrados*, ou seja, contêm a mesma quantidade de avaliações positivas e negativas.

Este notebook usa [`tf.keras`](https://www.tensorflow.org/guide/keras), uma API de alto nível para criar e treinar modelos no TensorFlow, além de [`tensorflow_hub`](https://www.tensorflow.org/hub), uma biblioteca para carregar modelos treinados do [TFHub](https://tfhub.dev) com uma única linha de código. Se quiser ver um tutorial de classificação de texto usando `tf.keras` mais avançado, confira o [Guia de Classificação de texto do MLCC](https://developers.google.com/machine-learning/guides/text-classification/).

In [ ]:
!pip install tensorflow-hub
!pip install tensorflow-datasets

In [ ]:
import os
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

## Baixe o dataset do IMDB

O dataset do IMDB está disponível em [imdb_reviews](https://www.tensorflow.org/datasets/catalog/imdb_reviews) ou nos [datasets do TensorFlow](https://www.tensorflow.org/datasets). O seguinte código baixa o dataset do IMDB para sua máquina (ou para o runtime do Colab):

In [ ]:
# Split the training set into 60% and 40% to end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

## Explore os dados

Vamos reservar um momento para compreender o formato dos dados. Cada exemplo é uma frase representando a avaliação do filme e um rótulo correspondente. A frase não é pré-processada de nenhuma forma. O rótulo é um número inteiro igual a 0 ou 1, em que 0 é uma avaliação negativa e 1, uma avaliação positiva.

Vamos exibir os primeiros 10 exemplos.

In [ ]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

Vamos imprimir também os primeiros 10 rótulos.

In [ ]:
train_labels_batch

## Crie o modelo

A rede neural é criada empilhando-se camadas, o que requer três decisões de arquitetura principais:

- Como representar o texto?
- Quantas camadas usar no modelo?
- Quantas *unidades ocultas* usar em cada camada?

Neste exemplo, os dados de entrada são frases. Os rótulos a serem previstos são 0 ou 1.

Uma maneira de representar o texto é converter as frases em vetores de embeddings. Use um embedding de texto pré-treinado como a primeira camada, o que traz três vantagens:

- Você não precisa se preocupar com o pré-processamento do texto.
- Você se beneficia do aprendizado por transferência.
- O embedding tem um tamanho fixo, então é mais simples de processar.

Neste exemplo, você usará um **modelo de embedding de texto pré-treinado** do [TensorFlow Hub](https://tfhub.dev) chamado [google/nnlm-en-dim50/2](https://tfhub.dev/google/nnlm-en-dim50/2).

Há diversos outros embeddings de texto pré-treinados do TFHub que podem ser usados neste tutorial:

- [google/nnlm-en-dim128/2](https://tfhub.dev/google/nnlm-en-dim128/2) – treinado com a mesma arquitetura NNLM nos mesmos dados que [google/nnlm-en-dim50/2](https://tfhub.dev/google/nnlm-en-dim50/2), mas com uma dimensão de embedding maior. Embeddings com dimensão maior podem melhorar a tarefa, mas pode levar mais tempo para treinar o modelo.
- [google/nnlm-en-dim128-with-normalization/2](https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2) – igual a [google/nnlm-en-dim128/2](https://tfhub.dev/google/nnlm-en-dim128/2), mas com uma normalização de texto adicional, como a remoção de pontuações. Isso pode ajudar se o texto da tarefa contiver caracteres adicionais ou pontuações.
- [google/universal-sentence-encoder/4](https://tfhub.dev/google/universal-sentence-encoder/4) – um modelo muito maior, com embeddings de 512 dimensões, treinado com um encoder de DAN (Deep Averating Network).

E muitos outros! Confira mais [modelos de embedding de texto](https://tfhub.dev/s?module-type=text-embedding) no TFHub.

Vamos criar uma camada do Keras que use um modelo do TensorFlow Hub para fazer o embedding das frases e testar em alguns exemplos de entrada. Não importa qual seja o tamanho do texto de entrada, o formato da saída dos embeddings será: `(num_examples, embedding_dimension)`.

In [ ]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

Agora, vamos criar o modelo completo:

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

As camadas são empilhadas sequencialmente para construir o classificador:

1. A primeira é uma camada do TensorFlow Hub. Essa camada usa um SavedModel pré-treinado para mapear uma frase em seu vetor de embeddings. O modelo de embedding de texto pré-treinado que você está usando ([google/nnlm-en-dim50/2](https://tfhub.dev/google/nnlm-en-dim50/2)) divide a frase em tokens, faz o embedding de cada token e depois combina o embedding. As dimensões resultantes são: `(num_examples, embedding_dimension)`. Para este modelo NNML, a dimensão do embedding `embedding_dimension` é 50.
2. O vetor de saída com tamanho fixo é passado por uma camada (`Dense`) totalmente conectada com 16 unidades ocultas.
3. A última camada é densamente conectada com um único nó de saída.

Vamos compilar o modelo.

### Função de perda e otimizador

Todo modelo precisa de uma função de perda e um otimizador para o treinamento. Como este é um problema de classificação binária e o modelo gera como saída logits (uma camada de unidade única com uma ativação linear), você usará a função de perda `binary_crossentropy`.

Essa não é a única opção para a função de perda. Por exemplo, você pode optar por `mean_squared_error`. Porém, de forma geral, `binary_crossentropy` lida melhor com probabilidades, pois mede a "distância" entre distribuições de probabilidade ou, no nosso caso, entre a distribuição de verdade fundamental (ground-truth) e as previsões.

Depois, quando você lidar com problemas de regressão (por exemplo, para prever o preço de uma casa), verá como usar outra função de perda chamada Erro Quadrático Médio.

Agora, configure o modelo para usar um otimizador e uma função de perda:

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

## Treine o modelo

Treine o modelo com 10 épocas em minilotes com 512 amostras. São feitas 10 iterações em todas as amostras nos tensores `x_train` e `y_train`. Durante o treinamento, monitore a perda e a exatidão do modelo para as 10 mil amostras do conjunto de validação:

In [ ]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

## Avalie o modelo

Vamos conferir o desempenho do modelo. Serão retornados dois valores: perda (um número que representa o erro; quanto menor, melhor) e exatidão.

In [ ]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

Essa estratégia bem simples atinge uma exatidão de cerca de 87%. Com estratégias mais avançadas, a exatidão do modelo deve se aproximar de 95%.

## Mais recursos

- Para ver uma maneira geral de trabalhar com strings como entradas e para ver uma análise detalhada do progresso da exatidão e da perda durante o treinamento, confira o tutorial [Classificação de texto com texto pré-processado](./text_classification.ipynb).
- Confira outros [tutoriais relacionados a texto](https://www.tensorflow.org/hub/tutorials#text-related-tutorials) que usam modelos treinados no TFHub.